In [1]:
import pandas as pd
import numpy as np
import h2o

# Machine Learning - Using h2o Auto ML

In [2]:
full_table = pd.read_csv('./data/processed_results/osc_df')

In [3]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: java version "1.8.0_231"; Java(TM) SE Runtime Environment (build 1.8.0_231-b11); Java HotSpot(TM) 64-Bit Server VM (build 25.231-b11, mixed mode)
  Starting server from /Users/nickparker/anaconda3/envs/DistributedComputing/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/kj/kg5zhyt51z7__kkfrysh9lv00000gn/T/tmpcty3wcnu
  JVM stdout: /var/folders/kj/kg5zhyt51z7__kkfrysh9lv00000gn/T/tmpcty3wcnu/h2o_nickparker_started_from_python.out
  JVM stderr: /var/folders/kj/kg5zhyt51z7__kkfrysh9lv00000gn/T/tmpcty3wcnu/h2o_nickparker_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,02 secs
H2O cluster timezone:,America/Los_Angeles
H2O data parsing timezone:,UTC
H2O cluster version:,3.28.0.2
H2O cluster version age:,1 day
H2O cluster name:,H2O_from_python_nickparker_vuzzy5
H2O cluster total nodes:,1
H2O cluster free memory:,3.556 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"accepting new members, healthy"


### Train/Test Split

First Year of Existance:
- gg_drama 1943
- gg_comedy 1951
- pga 1989
- bafta 1960
- dga 1948
- sag 1995
- cannes 1970

In [4]:
# I pick a min_year where the awards shows will be relevant
min_year = 1989

# Leftover code from designing the split
# # 80/20 Split (random)
# table_sized = full_table.loc[(full_table['year'] >= min_year)]
# rand_rows = np.random.choice(table_sized.shape[0], table_sized.shape[0])
# split = int(0.8 * len(rand_rows))
# train = table_sized.iloc[rand_rows[:split]]
# valid = table_sized.iloc[rand_rows[split:]]

# # 80/20 Split (sequential)
# year_range = 2019 - min_year
# train = full_table.loc[((full_table['year'] <= 2019 - 0.2 * year_range) & (full_table['year'] >= min_year))]
# valid = full_table.loc[((full_table['year'] > 2019 - 0.2 * year_range) & (full_table['year'] < 2019))]

# # Final Prediction
# train = full_table.loc[((full_table['year'] < 2019) & (full_table['year'] >= min_year))]
# test = full_table.loc[(full_table['year'] == 2019)]

# train.to_csv('./data/processed_results/train.csv', index = False)
# valid.to_csv('./data/processed_results/valid.csv', index = False)

# H2O's Auto ML

## Pick best model by validating with the first 80% of the data

In [5]:
# 80/20 Split (sequential) Excluding 2019
year_range = 2019 - min_year
train = full_table.loc[((full_table['year'] <= 2019 - 0.2 * year_range) & (full_table['year'] >= min_year))]
valid = full_table.loc[((full_table['year'] > 2019 - 0.2 * year_range) & (full_table['year'] < 2019))]

# train.to_csv('./data/processed_results/train.csv', index = False)
# valid.to_csv('./data/processed_results/valid.csv', index = False)

In [6]:
train.columns

Index(['Unnamed: 0', 'year', 'film', 'wiki', 'winner', 'Oscar_win',
       'nom_gg_drama', 'winner_gg_drama', 'nom_gg_comedy', 'winner_gg_comedy',
       'nom_pga', 'winner_pga', 'nom_bafta', 'winner_bafta', 'nom_dga',
       'winner_dga', 'nom_sag', 'winner_sag', 'nom_cannes', 'winner_cannes'],
      dtype='object')

In [7]:
from h2o.automl import H2OAutoML, get_leaderboard

# Import a sample binary outcome train/test set into H2O
train = h2o.H2OFrame(train)
test = h2o.H2OFrame(valid)

# Identify predictors and response
predictors = ['year','nom_gg_drama', 'winner_gg_drama', 'nom_gg_comedy', 'winner_gg_comedy',
       'nom_pga', 'winner_pga', 'nom_bafta', 'winner_bafta', 'nom_dga',
       'winner_dga', 'nom_sag', 'winner_sag', 'nom_cannes', 'winner_cannes']

x = predictors
y = 'Oscar_win'

# For binary classification, response should be a factor
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

# Run AutoML for 20 base models (limited to 1 hour max runtime by default)
aml = H2OAutoML(max_models=20, seed=1)
aml.train(x=x, y=y, training_frame=train)

# AutoML Leaderboard
lb = aml.leaderboard

# Optionally edd extra model information to the leaderboard
lb = get_leaderboard(aml, extra_columns='ALL')

# Print all rows (instead of default 10 rows)
lb.head(rows=lb.nrows)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
AutoML progress: |███████████
22:13:25.124: Skipping training of model GBM_5_AutoML_20200121_221310 due to exception: water.exceptions.H2OModelBuilderIllegalArgumentException: Illegal argument(s) for GBM model: GBM_5_AutoML_20200121_221310.  Details: ERRR on field: _min_rows: The dataset size is too small to split for min_rows=100.0: must have at least 200.0 (weighted) rows, but have only 147.0.


█████████████████████████████████████████████| 100%


model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse,training_time_ms,predict_time_per_row_ms
DeepLearning_1_AutoML_20200121_221310,0.939016,0.25691,0.75449,0.172295,0.28097,0.0789442,51,0.029673
XGBoost_grid__1_AutoML_20200121_221310_model_4,0.881639,0.4097,0.419895,0.102459,0.37539,0.140918,205,0.015534
StackedEnsemble_AllModels_AutoML_20200121_221310,0.872131,0.305983,0.666593,0.188197,0.29735,0.088417,148,0.357138
StackedEnsemble_BestOfFamily_AutoML_20200121_221310,0.856066,0.285115,0.639881,0.176393,0.279779,0.0782763,244,0.074173
XGBoost_grid__1_AutoML_20200121_221310_model_2,0.834918,0.446651,0.281636,0.102459,0.38018,0.144537,179,0.035899
XGBoost_grid__1_AutoML_20200121_221310_model_1,0.826066,0.603947,0.327991,0.147049,0.428488,0.183602,300,0.020217
DRF_1_AutoML_20200121_221310,0.652295,0.641437,0.231538,0.330492,0.410925,0.16886,80,0.032412
XRT_1_AutoML_20200121_221310,0.633934,0.646703,0.228248,0.363279,0.41381,0.171239,92,0.024009
XGBoost_3_AutoML_20200121_221310,0.510984,0.564226,0.159188,0.303279,0.413365,0.170871,204,0.030723
GLM_1_AutoML_20200121_221310,0.481967,0.589107,0.188002,0.495902,0.402091,0.161677,43,0.015254


## Once best model is picked, retrain that on the entire dataset

In [8]:
top_model = aml.leader
top_model

Model Details
H2ODeepLearningEstimator :  Deep Learning
Model Key:  DeepLearning_1_AutoML_20200121_221310


Status of Neuron Layers: predicting Oscar_win, 2-class classification, bernoulli distribution, CrossEntropy loss, 402 weights/biases, 10.7 KB, 1,646 training samples, mini-batch size 1


,,layer,units,type,dropout,l1,l2,mean_rate,rate_rms,momentum,mean_weight,weight_rms,mean_bias,bias_rms
0,,1,15,Input,0,,,,,,,,,
1,,2,10,Rectifier,0,0,0,0.00150175,0.00120049,0,-0.0111631,0.258635,0.508221,0.0384297
2,,3,10,Rectifier,0,0,0,0.00102098,0.000600193,0,-0.0239444,0.317506,0.992031,0.0863466
3,,4,10,Rectifier,0,0,0,0.00461687,0.0100446,0,-0.0126535,0.345424,1.00641,0.092561
4,,5,2,Softmax,,0,0,0.00156279,0.00123843,0,-0.532108,1.48643,1.82675e-05,0.0664708




ModelMetricsBinomial: deeplearning
** Reported on train data. **

MSE: 0.05962855633706177
RMSE: 0.2441895909678825
LogLoss: 0.21462630482073308
Mean Per-Class Error: 0.10918032786885246
AUC: 0.9400000000000001
AUCPR: 0.7875543747981884
Gini: 0.8800000000000001

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.5005644230033649: 


,,0,1,Error,Rate
0,0,119.0,3.0,0.0246,(3.0/122.0)
1,1,6.0,19.0,0.24,(6.0/25.0)
2,Total,125.0,22.0,0.0612,(9.0/147.0)



Maximum Metrics: Maximum metrics at their respective thresholds


...

,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,accuracy,0.56666666,0.5217492,0.8333333,0.0,0.0,1.0,1.0
1,auc,0.9585921,0.07172053,0.8757764,NaN,NaN,1.0,1.0
2,aucpr,0.30644923,0.5307857,0.91934776,NaN,NaN,0.0,0.0
3,err,0.13479064,0.16769698,0.16666667,0.40675125,0.1005353,0.0,0.0
4,err_count,1.6666666,2.8867514,5.0,NaN,NaN,0.0,0.0
5,f0point5,0.5978224,0.48785332,0.8518519,0.11273263,0.02452778,1.0,1.0
6,f1,0.96732026,0.056602966,0.9019608,NaN,NaN,1.0,1.0
7,f2,-Infinity,NaN,0.9583333,-Infinity,-Infinity,1.0,1.0
8,lift_top_group,11.959344,15.562046,1.3043479,0.3357568,0.15661347,29.0,29.0
9,logloss,0.15290837,0.19466485,0.47597873,None,None,0.13219051,0.15637262



See the whole table with table.as_data_frame()

Scoring History: 


,,timestamp,duration,training_speed,epochs,iterations,samples,training_rmse,training_logloss,training_r2,training_auc,training_pr_auc,training_lift,training_classification_error
0,,2020-01-21 22:13:25,0.000 sec,None,0.000000,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,,2020-01-21 22:13:25,0.495 sec,48000 obs/sec,0.979592,1,144.0,0.408663,0.519302,-0.183223,0.808852,0.486066,5.88,0.183673
2,,2020-01-21 22:13:25,0.529 sec,54866 obs/sec,11.197279,11,1646.0,0.244190,0.214626,0.577537,0.940000,0.787554,5.88,0.061224



Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,winner_gg_drama,1.000000,1.000000,0.088492
1,nom_dga,0.968969,0.968969,0.085746
2,nom_cannes,0.868818,0.868818,0.076883
3,nom_sag,0.862101,0.862101,0.076289
4,winner_cannes,0.860334,0.860334,0.076133
5,winner_pga,0.834897,0.834897,0.073882
6,nom_gg_comedy,0.832120,0.832120,0.073636
7,nom_bafta,0.814545,0.814545,0.072081
8,nom_gg_drama,0.721495,0.721495,0.063847
9,winner_bafta,0.704079,0.704079,0.062305


In [9]:
# Re-split data on min_year to 2019
train = full_table.loc[((full_table['year'] < 2019) & (full_table['year'] >= min_year))]
test = full_table.loc[(full_table['year'] == 2019)]

# Import a binary outcome train/test set into H2O
train = h2o.H2OFrame(train)
test = h2o.H2OFrame(test)

# For binary classification, response should be a factor
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [10]:
top_model.train(x=x, y=y, training_frame=train)

preds = top_model.predict(test)

preds

deeplearning Model Build progress: |██████████████████████████████████████| 100%
deeplearning prediction progress: |███████████████████████████████████████| 100%


predict,p0,p1
0,0.999997,3.27692e-06
0,0.998913,0.00108672
0,0.999999,6.39159e-07
0,0.999904,9.57657e-05
0,0.999997,3.27692e-06
0,0.999968,3.22032e-05
0,0.853322,0.146678
0,0.999507,0.000493258
0,0.998038,0.0019618


## Predict the winner

In [11]:
test['pred'] = preds['predict']
test['probA'] = preds['p1']
test_pd = test.as_data_frame(use_pandas=True)

In [12]:
test_pd[['film','probA']].sort_values('probA', ascending = False)

,film,probA
6,1917 (2019 film),1.466781e-01
8,Parasite (2019 film),1.961801e-03
1,The Irishman,1.086720e-03
7,Once Upon a Time in Hollywood,4.932576e-04
3,Joker (2019 film),9.576566e-05
5,Marriage Story,3.220317e-05
0,Ford v Ferrari,3.276917e-06
4,Little Women (2019 film),3.276917e-06
2,Jojo Rabbit,6.391595e-07


In [13]:
test_pd.T # The DGA's and BAFTA's Haven't come out yet

,0,1,2,3,4,5,6,7,8
Unnamed: 0,554,555,556,557,558,559,560,561,562
year,2019,2019,2019,2019,2019,2019,2019,2019,2019
film,Ford v Ferrari,The Irishman,Jojo Rabbit,Joker (2019 film),Little Women (2019 film),Marriage Story,1917 (2019 film),Once Upon a Time in Hollywood,Parasite (2019 film)
wiki,/wiki/Ford_v_Ferrari,/wiki/The_Irishman,/wiki/Jojo_Rabbit,/wiki/Joker_(2019_film),/wiki/Little_Women_(2019_film),/wiki/Marriage_Story,/wiki/1917_(2019_film),/wiki/Once_Upon_a_Time_in_Hollywood,/wiki/Parasite_(2019_film)
winner,False,False,False,False,False,False,False,False,False
Oscar_win,0,0,0,0,0,0,0,0,0
nom_gg_drama,0,1,0,1,0,1,1,0,0
winner_gg_drama,0,0,0,0,0,0,1,0,0
nom_gg_comedy,0,0,1,0,0,0,0,1,0
winner_gg_comedy,0,0,0,0,0,0,0,1,0
